In [1]:
from time import sleep

In [2]:
from pinkylib import IMU

imu = IMU()

ModuleNotFoundError: No module named 'pinkylib'

In [ ]:
try:
    while True:
        # 가속도 데이터 읽기
        accel_data = imu.get_accel()
        print(f"Accelerometer: Ax={accel_data[0]}, Ay={accel_data[1]}, Az={accel_data[2]}")
            
        # 자이로 데이터 읽기
        gyro_data = imu.get_gyro()
        print(f"Gyroscope: Gx={gyro_data[0]}, Gy={gyro_data[1]}, Gz={gyro_data[2]}")
            
        # 주기적인 딜레이
        sleep(1)

except KeyboardInterrupt:
    # 사용자 인터럽트 시 종료
    print("Process stopped by user")

finally:
    # SMBus 자원 해제
    imu.bus.close()

In [ ]:
from smbus2 import SMBus
from time import sleep

# MPU6500 레지스터와 주소
PWR_MGMT_1   = 0x6B
SMPLRT_DIV   = 0x19
CONFIG       = 0x1A
GYRO_CONFIG  = 0x1B
INT_ENABLE   = 0x38
ACCEL_XOUT_H = 0x3B
ACCEL_YOUT_H = 0x3D
ACCEL_ZOUT_H = 0x3F
GYRO_XOUT_H  = 0x43
GYRO_YOUT_H  = 0x45
GYRO_ZOUT_H  = 0x47

In [ ]:
class IMU:
    def __init__(self, bus_num=1, device_address=0x68):
        # I2C 버스를 열고 특정 장치 주소와 통신을 설정
        self.bus = SMBus(bus_num)
        self.device_address = device_address
        self.init_device()  # 장치 초기화

    def init_device(self):
        # 샘플레이트 분주기 레지스터에 값 쓰기 (분주기 설정)
        self.bus.write_byte_data(self.device_address, SMPLRT_DIV, 7)
        # 전원 관리 1 레지스터에 값 쓰기 (장치 활성화)
        self.bus.write_byte_data(self.device_address, PWR_MGMT_1, 1)
        # 설정 레지스터에 값 쓰기 (DLPF: 대역폭 설정)
        self.bus.write_byte_data(self.device_address, CONFIG, 0)
        # 자이로 범위 설정 (±2000dps)
        self.bus.write_byte_data(self.device_address, GYRO_CONFIG, 24)
        # 인터럽트 활성화 설정
        self.bus.write_byte_data(self.device_address, INT_ENABLE, 1)

    def read_raw_data(self, addr):
        # 지정된 주소의 상위 바이트 읽기
        high = self.bus.read_byte_data(self.device_address, addr)
        # 지정된 주소의 다음 바이트(하위 바이트) 읽기
        low = self.bus.read_byte_data(self.device_address, addr + 1)
        
        # 상위 바이트와 하위 바이트를 하나의 16비트 값으로 만들기
        value = (high << 8) | low
        
        # 16비트 정수로 변환 (부호 있는 값으로 범위 조정)
        if value > 32768:
            value -= 65536
        return value
        
    def get_accel(self):
        # 각 가속도 축에 대한 데이터 읽기
        acc_x = self.read_raw_data(ACCEL_XOUT_H)
        acc_y = self.read_raw_data(ACCEL_YOUT_H)
        acc_z = self.read_raw_data(ACCEL_ZOUT_H)
        
        # 가속도 값을 중력 g 단위로 변환
        Ax = round(acc_x / 16384.0, 2)
        Ay = round(acc_y / 16384.0, 2)
        Az = round(acc_z / 16384.0, 2)
        
        return Ax, Ay, Az
        
    def get_gyro(self):
        # 각 자이로 축에 대한 데이터 읽기
        gyro_x = self.read_raw_data(GYRO_XOUT_H)
        gyro_y = self.read_raw_data(GYRO_YOUT_H)
        gyro_z = self.read_raw_data(GYRO_ZOUT_H)
        
        # 자이로 값을 각속도 dps 단위로 변환
        Gx = round(gyro_x / 131.0, 2)
        Gy = round(gyro_y / 131.0, 2)
        Gz = round(gyro_z / 131.0, 2)
        
        return Gx, Gy, Gz
        
    def get_accel_gyro_data(self):
        # 가속도 원시 데이터 읽기
        acc_x = self.read_raw_data(ACCEL_XOUT_H)
        acc_y = self.read_raw_data(ACCEL_YOUT_H)
        acc_z = self.read_raw_data(ACCEL_ZOUT_H)
        
        # 자이로 원시 데이터 읽기
        gyro_x = self.read_raw_data(GYRO_XOUT_H)
        gyro_y = self.read_raw_data(GYRO_YOUT_H)
        gyro_z = self.read_raw_data(GYRO_ZOUT_H)

        # 각 축에 대해 변환과 함께 가속도와 자이로 데이터 계산
        Ax = acc_x / 16384.0  # X축 가속도[g]
        Ay = acc_y / 16384.0  # Y축 가속도[g]
        Az = acc_z / 16384.0  # Z축 가속도[g]

        Gx = gyro_x / 131.0  # X축 자이로[dps]
        Gy = gyro_y / 131.0  # Y축 자이로[dps]
        Gz = gyro_z / 131.0  # Z축 자이로[dps]

        # 결과를 딕셔너리 형태로 반환
        return {"Ax": Ax, "Ay": Ay, "Az": Az, "Gx": Gx, "Gy": Gy, "Gz": Gz}

In [ ]:
imu = IMU()

In [ ]:
try:
    while True:
        # 가속도 데이터 읽기
        accel_data = imu.get_accel()
        print(f"Accelerometer: Ax={accel_data[0]}, Ay={accel_data[1]}, Az={accel_data[2]}")
            
        # 자이로 데이터 읽기
        gyro_data = imu.get_gyro()
        print(f"Gyroscope: Gx={gyro_data[0]}, Gy={gyro_data[1]}, Gz={gyro_data[2]}")
            
        # 주기적인 딜레이
        sleep(1)

except KeyboardInterrupt:
    # 사용자 인터럽트 시 종료
    print("Process stopped by user")

finally:
    # SMBus 자원 해제
    imu.bus.close()